In [5]:
import pandas as pd
csv = pd.read_csv("IMDB Top 250 Movies.csv")
arr = csv.iloc[:,1]
arr = list(arr)
arr

['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 'The Godfather Part II',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Good, the Bad and the Ugly',
 'Forrest Gump',
 'Fight Club',
 'The Lord of the Rings: The Two Towers',
 'Inception',
 'Star Wars: Episode V - The Empire Strikes Back',
 'The Matrix',
 'Goodfellas',
 "One Flew Over the Cuckoo's Nest",
 'Se7en',
 'Seven Samurai',
 "It's a Wonderful Life",
 'The Silence of the Lambs',
 'City of God',
 'Saving Private Ryan',
 'Interstellar',
 'Life Is Beautiful',
 'The Green Mile',
 'Star Wars: Episode IV - A New Hope',
 'Terminator 2: Judgment Day',
 'Back to the Future',
 'Spirited Away',
 'The Pianist',
 'Psycho',
 'Parasite',
 'Léon: The Professional',
 'The Lion King',
 'Gladiator',
 'American History X',
 'The Departed',
 'The Usual Suspects',
 'The Prestige',
 'Whiplash',
 'Casablanca',
 'Grave o